# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
%pip install pymongo

In [1]:
import pymongo
import pandas as pd
from pymongo import MongoClient
client = MongoClient()

In [2]:
db= client.companies

In [8]:
collect= db.companies

In [9]:
res= collect.find()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [12]:
query = {"name":"Babelgum"}
select = {"name":True, "_id": False}
res= collect.find(query,select)

list(res)

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [13]:
query = {"number_of_employees": {"$gt": 5000}}
select = {"name": True, "_id": False, "number_of_employees": True}

res = collect.find(query, select).sort("number_of_employees", -1).limit(5)

list(res)

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [14]:
query = {"$and": [{"founded_year": {"$gte": 2000}}, {"founded_year":{"$lte":2005}}]}
select = {"_id": False, "founded_year": True, "name": True}


res = collect.find(query, select).limit(5
                                       )
list(res)

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [15]:
query = {"$and": [{"ipo.valuation_amount":{"$gt": 100000000}}, {"founded_year":{"$lt":2010}}]}
select= {"name":True, "valuation_amount": True, "_id": False} 
res = collect.find(query, select).limit(5)

list(res)

[{'name': 'Facebook'},
 {'name': 'Twitter'},
 {'name': 'Yelp'},
 {'name': 'LinkedIn'},
 {'name': 'Amazon'}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [16]:
query={"$and": [{"number_of_employees":{"$lt": 1000}}, {"founded_year": {"$lt":2005}}]}
select={"name": True,"number_of_employees": True, "_id":False}
res = collect.find(query, select).sort("number_of_employees", pymongo.DESCENDING).limit(5)

list(res)

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Webmetrics', 'number_of_employees': 900}]

### 6. All the companies that don't include the `partners` field.

In [19]:
query = {"partners": {"$exists": False}}
select = {"name" : True, "_id":False, "partners":True}
res = collect.find(query, select)

list(res)

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [20]:
query = { "category_code": {"$type": "null"}}
select = {"name":True, "_id": False}
res = collect.find(query, select).limit(5)

list(res)

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [21]:
query = {"$and": [{"number_of_employees": {"$gte":100}}, 
                  {"number_of_employees" : {"$lt":1000}}]}
select = {"name":True, "number_of_employees": True, "_id": False}
res = collect.find(query, select).limit(5)

list(res)

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [22]:
query = {"ipo.valuation_amount": {"$gte": 0}}
select={"name":True, "ipo.valuation_amount":True, "_id": False}
res = collect.find(query, select).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(5) #no acabo de pillar el sort de mongo

list(res)# Your Code

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [24]:
pipeline = [
    {"$group": {"_id": "$company", "total_employees": {"$sum": "$employees"}}},
    {"$sort": {"total_employees": -1}},
    {"$limit": 10}
]

# Ejecuta la pipeline de agregación
results = list(collect.aggregate(pipeline))

# Imprime los resultados
for result in results:
    print(result["_id"], result["total_employees"])

None 0


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [25]:
query = {"founded_month":{"$gte":6}}  
select = {"founded_month":True, "name": True, "_id" : False} 
res = collect.find(query, select).limit(10) #ponia 1000 pero por evitar el scroll innecesario

list(res)

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [26]:
query = {"$and":[{"founded_year": {"$lt": 2000}}, 
                 {"acquisition.price_amount": {"$gt": 10000000}}]}
select = {"name": True, "acquisition.price_amount": True, "_id": False}
res = collect.find(query, select).limit(5)

list(res)

[{'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep', 'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar', 'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish', 'acquisition': {'price_amount': 300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [41]:
query = {"acquisition.acquired_year": {"$gt": 2010}}
select = {"name": True, "_id": False}
res = collect.find(query, select)

names = []
for result in results:
    names.append(result["name"])
print(names)

KeyError: 'name'

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [42]:
query = {}
select = {"name":True, "founded_year": True, "_id":False}
res = collect.find(query, select).sort("founded_year", pymongo.DESCENDING).limit(5)

list(res)

[{'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [30]:
query = {"founded_day": {"$lt":7}}
select = {"name": True, "founded_day": True, "acquisition.price_amount": True, "_id":False}
res = collect.find(query, select).sort("acquisition.price_amount", pymongo.DESCENDING).limit(5)

list(res)

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
query = {"$and":[{"category_code": "web"},{"number_of_employees":{"$gt":4000}}]}
select= {"name":True, "category_code":True, "number_of_employees":True, "_id":False}
res = collect.find(query, select).sort("number_of_employees", pymongo.ASCENDING).limit(5)

list(res)

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [44]:
query = {"$and":[{"acquisition.price_amount":{"$gt": 10000000}},
                 {"acquisition.price_currency_code": "EUR"}]}
select = {"name": True, "acquisition.price_amount": True, "_id":False}
res = collect.find(query, select).limit(5)

list(res)

[{'name': 'ZYB', 'acquisition': {'price_amount': 31500000}},
 {'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Greenfield Online', 'acquisition': {'price_amount': 40000000}},
 {'name': 'Webedia', 'acquisition': {'price_amount': 70000000}},
 {'name': 'Wayfinder', 'acquisition': {'price_amount': 24000000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [46]:
query = {"acquisition.acquired_month":{"$lte": 3}}
select = {"name":True, "acquisition":True, "_id": False}
res = collect.find(query, select).limit(10) #pide 10 pero era mucho texto
# names = []
# for result in results:
#     names.append(result["name"])
# print(names)

list(res)

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code